# QuantumSim - Computational Improvements

Author: Wouter Pennings<br>
Date: April 2025

This notebook outlines proposed improvements to the modeling approach used in QuantumSim.

It focuses on addressing two primary issues:
- Memory limitations
- Poor performance

Summary of proposed changes to QuantumSim:
- Use of sparse matrices and efficient algorithm implementations
- Integration of native code via JIT (Just-In-Time) compilation
- Hardware acceleration using GPUs
- Caching and memoization techniques
- Lazy generation of matrices
- Minor additional enhancements, such as:
  - Switching the state vector to a row-based format

These changes aim to significantly improve the performance of QuantumSim while maintaining the same number of qubits. They also open the door to simulating a greater number of qubits—potentially a dozen or more.

Toward the end of this notebook, performance and output comparisons will be made between this improved version and the current implementation of QuantumSim.

The full implementation with all proposed improvements can be found in [quantumsim_performante.py](quantumsim_performante.py).

Note: The implementations presented here are not necessarily optimized. Their purpose is to demonstrate that such improvements can effectively address the identified issues. This enhanced version of QuantumSim builds upon the minimal implementation introduced in [QuantumSimIntroduction.ipynb](QuantumSimIntroduction.ipynb).


In [3]:
# Imports
import math
import cmath
import time
from typing import List, Union, Tuple
import numpy as np
import scipy.sparse as sparse
from numba import njit
try:
    import cupy
    import cupyx.scipy.sparse as cupysparse
    GPU_AVAILABLE = True
except:
    print("[ERROR] Cupy could not be imported, make sure that your have installed Cupy")
    print("\tIf you do not have a NVIDIA GPU, you cannot install Cupy")
    print("\tQuantumsim will still work accordingly, just less performant")
    print("\t > Installation guide: https://docs.cupy.dev/en/stable/install.html")
    GPU_AVAILABLE = False

[ERROR] Cupy could not be imported, make sure that your have installed Cupy
	If you do not have a NVIDIA GPU, you cannot install Cupy
	Quantumsim will still work accordingly, just less performant
	 > Installation guide: https://docs.cupy.dev/en/stable/install.html


## Inherit quantum computer simulation problem

Each additional qubit that a quantum computer/circuit has doubles its computing power, this "property" of quantum computers is what makes them exponentially faster than classical computers. However, it is also the reason why they are difficult to simulate, because each additional qubit requires your computer to be twice as powerful.

## Computationally heavy operations

There are two main (mathematical) operations inside of Quantumsim which combined take up the vast majority of the computational time. They are discussed below.

### Kronecker product
Kronecker product is used to build the unitary operation matrices. The time complexity of the function is $O(4^n)$, which means that every additional qubit, makes it four times slower. Building a unitary operation matrix takes roughly 20 seconds 

All the kronecker operations combined, take roughly 20 seconds for one operation in a fifteen qubits circuit. The kronecker product can be found in `CircuitUnitaryOperation.get_combined_operation_for_qubit` and `CircuitUnitaryOperation.get_combined_operation_for_cnot`

```python
def get_combined_operation_for_qubit(operation:np.ndarray, q:int, N:int):
    identity = QubitUnitaryOperation.get_identity()
    combined_operation = np.eye(1,1)

    for i in range(0, N):
        if i == q:
            combined_operation = np.kron(combined_operation, operation)
        else:
            combined_operation = np.kron(combined_operation, identity)

    return combined_operation
```

### Matrix-vector multiplication

Matrix-vector multiplication is use to "execute" the unitary operation matrices on the `Statevector`. The time complexity of the matrix-vector multiplication is $O(N^2)$. Matrix-vector multiplication between a operation and the statevector in a 15 qubit system takes around four second. The multiplication can be found in `Statevector.apply_unitary_operation`  

The `apply_unitary_operation` also has a check whether the operation is a unitary matrix 

```python
def apply_unitary_operation(self, operation):
    # Check if operation is a unitary matrix
    if not np.allclose(np.eye(2**self.N), np.conj(operation.T) @ operation):
        raise ValueError("Input matrix is not unitary")
    
    self.state_vector = operation @ self.state_vector
```

## Memory limitations

An a regular 16 GB system there currently is a ceiling of 14 qubits. The unitary matrices (operations) have a space complexity of $O(4^n)$, meaning that every additional qubit quadruples the size of the unitary matrices. 

Amount of elements in a unitary matrix from 14 qubit circuit:

$(2^{14})^2 = 268,435,456\ elements$

Every elements uses 128 bits or 16 bytes, as it is a complex numbers of with the real and imaginary part both are 64 bit floats.

$268,435,456 * 16 = 4,294,967,296\ bytes$

If we do the same calculations for a 15 qubit system, we can see that we are already out of memory in a 16 GB system. These are the elements and bytes for a 15 qubit system:

$(2^{15})^2 = 1,073,741,824\ elements$

$1,073,741,824 * 16 = 17,179,869,184\ bytes$

## Proposed Changes & Improvements

### Sparse matrix (COO)

Unitary operation matrices are mostly sparse, meaning that most elements in the matrix are zero. **Sparsity** refers to the percentage of **non-zero (NNZ) elements** in the matrix. As the number of qubits in a quantum circuit increases, the corresponding operation matrices grow larger and become increasingly sparse.  

The sparsity of a matrix is defined as the percentage of NNZ elements relative to the total number of elements in the matrix.  

For example, consider a **10×10** matrix with **7 non-zero elements (NNZ)**. The sparsity is calculated as follows:  

$$
\text{sparsity} = 100 \times \left( \frac{\text{NNZ}}{\text{total elements}} \right) = 100 \times \left( \frac{7}{10 \times 10} \right) = 100 \times 0.07 = 7\%
$$

To store these unitary matrices more efficiently by exploiting the sparsity of them. They are converted to a [sparse matrix](https://en.wikipedia.org/wiki/Sparse_matrix), these are special data structures to store sparse matrices very efficiently. There are many different types, but COO (Coordinate list) was chosen for Quantumsim. Taking a Kronecker product using the COO format is computationally extremely efficient, much better then E.G. CSR (Compressed sparse row)

The sparsity of a **Hadamard operation matrix** (the least sparse unitary matrix) can be determined using the following formula, where $n$ represents the number of qubits in the circuit:  

$$
f(n) = 100 \times \left(\frac{1}{2}\right)^{(n-1)}
$$

Additionally, the amount of memory required to store a **Hadamard operation matrix** for $n$ qubits can be estimated using these formulas:  

- **Dense matrix:** $f(n) = 4^n \times 16$
- **Sparse (COO) matrix:** $f(n) = 96 \times 2^{(n-1)}$

One thing to keep in mind is that once the circuit exceeds a certain number of qubits, the unitary operation matrices cannot be converted back to dense matrices because they will be larger than the amount of memory available. This specific number of qubits depends on the system, but is generally between 13 and 16 qubits. This means that all algorithms and functions involving a unitary operation matrix should be implemented to support sparse matrices instead of dense matrices; requiring a rewrite of a large part of the entire simulation.

**Scipy** provides sparse matrix functionality, data structures and functions, to work with (COO) sparse matrices.

The `Circuit` class changes from this dense implementation:

```python
import numpy as np

class Circuit:
    """
    Class representing a quantum circuit of N qubits.
    """
    def __init__(self, N):
        self.qubits = N
        self.state_vector = StateVector(self.qubits)
        self.quantum_states = [self.state_vector.get_quantum_state()]
        # Circuit.operations contains the unitary operation matrices in a dense (regular) form
        self.operations: list[np.ndarray] = [] 
        self.descriptions = []
```

to this sparse matrix implementation:

```python
import scipy as sc

class Circuit:
    """
    Class representing a quantum circuit of N qubits.
    """
    def __init__(self, N, use_lazy=False, use_cache=False, use_GPU=False):
        self.N = N
        self.state_vector = StateVector(self.N)
        self.quantum_states = [self.state_vector.get_quantum_state()]
        # Circuit.operations contains the unitary operation matrices in a sparse COO matrix
        # Data structure is provided by SciPy
        self.operations: list[sc.sparse.coo_matrix] = []
        self.descriptions = []
```

### Sparse matrix algorithms

Now that the operation matrices have been converted to sparse COO matrices, all algorithms involved have to be updated too. In addition to making the simulation work again but many possibility of many more qubits, the algorithms will also be much faster. Optimized mathematical operations such as, multiplications, transpose and kronecker, on sparse matrices are much faster than on dense matrices. The greater the sparsity greater the performance improvements.

Kronecker product and matrix-vector multiplication are the only two algoriths which will need to be converted for the COO format. Why they are used in QuantumSim can be found earlier in the notebook. Scipy already provides a sparse implementation for kronecker product; [scipy.sparse.kron(A, B)](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.kron.html). This implementation from SciPy is used in QuantumSim, but will be slightly altered to make it work. Because the "base" unitary operation matrices are 2x2, the SciPy implementation converts them to dense matrices, but that is not what is most optimal. They have probably done this for performance reasons, but in our specific case that will not work. Below you find the altered SciPy kronecker implementation:

In [ ]:
# Source: https://github.com/scipy/scipy/blob/v1.15.1/scipy/sparse/_construct.py#L458
# Docs: https://docs.scipy.org/doc/scipy-1.15.1/reference/generated/scipy.sparse.kron.html
def coo_kron(A:sparse.coo_matrix, B:sparse.coo_matrix) -> sparse.coo_matrix:
    output_shape = (A.shape[0] * B.shape[0], A.shape[1] * B.shape[1])

    if A.nnz == 0 or B.nnz == 0:
        # kronecker product is the zero matrix
        return sparse.coo_matrix(output_shape)

    # Expand entries of a into blocks
    # When using more then 32 qubits, increase to int64
    row = np.asarray(A.row, dtype=np.int32).repeat(B.nnz)
    col = np.asarray(A.col, dtype=np.int32).repeat(B.nnz)
    data = A.data.repeat(B.nnz)
    
    row *= B.shape[0]
    col *= B.shape[1]

    # increment block indices
    row = row.reshape(-1, B.nnz)
    row += B.row
    row = row.reshape(-1)

    col = col.reshape(-1, B.nnz)
    col += B.col
    col = col.reshape(-1)

    # compute block entries
    data = data.reshape(-1, B.nnz) * B.data
    data = data.reshape(-1)

    return sparse.coo_matrix((data, (row, col)), shape=output_shape)

Matrix vector multiplication for a sparse matrix is not common for the COO format. You can either convert to a format such as CSC, which is much more common for multiplications, or you can convert from COO to CSC, which is a time-consuming and computationally intensive process. In practice, the performance penalty for using COO is not large, and therefore converting to a more optimal format for multiplication is not worthwhile. 

The implementation for sparse matrix vector multiplication can be found below, it is not based on an existing implementation and is made for a column vector

In [ ]:
def coo_spmv_column(rowIdx:np.ndarray[np.int32], 
                    colIdx:np.ndarray[np.int32], 
                    values:np.ndarray[np.complex128], 
                    v:np.ndarray[np.complex128]) -> np.ndarray[np.complex128]:
    """
    Performs sparse matrix-vector (column based) multiplication using COO format.
    """
    out = np.zeros((len(v), 1), dtype=values.dtype)  # Initialize output vector
    nnz = len(values)  # Number of nonzero elements

    for i in range(nnz):  # Iterate over nonzero elements
        out[rowIdx[i], 0] += values[i] * v[colIdx[i], 0]

    return out

In [ ]:
@njit
def coo_spmv_row(rowIdx, colIdx, values, v):
    """
    Performs sparse matrix-vector (row based) multiplication using COO format.
    
    Parameters:
    - rowIdx (list[int]): Row indices of nonzero elements.
    - colIdx (list[int]): Column indices of nonzero elements.
    - values (list[float]): Nonzero values of the matrix.
    - v (numpy array): Dense vector for multiplication.
    
    Returns:
    - numpy array: Result vector y = A * v
    """
    out = np.zeros(len(v), dtype=values.dtype)  # Initialize output vector
    nnz = len(values)  # Number of nonzero elements

    for i in range(nnz):  # Iterate over nonzero elements
        out[rowIdx[i]] += values[i] * v[colIdx[i]]

    return out

In [ ]:
def coo_kron(A, B):
    output_shape = (A.shape[0] * B.shape[0], A.shape[1] * B.shape[1])

    if A.nnz == 0 or B.nnz == 0:
        # kronecker product is the zero matrix
        return sparse.coo_sparse(output_shape)

    # Expand entries of a into blocks
    # When using more then 32 qubits, increase to int64
    row = np.asarray(A.row, dtype=np.int32).repeat(B.nnz)
    col = np.asarray(A.col, dtype=np.int32).repeat(B.nnz)
    data = A.data.repeat(B.nnz)

    row *= B.shape[0]
    col *= B.shape[1]

    # increment block indices
    row = row.reshape(-1,B.nnz)
    row += B.row
    row = row.reshape(-1)

    col = col.reshape(-1,B.nnz)
    col += B.col
    col = col.reshape(-1)

    # compute block entries
    data = data.reshape(-1,B.nnz) * B.data
    data = data.reshape(-1)

    return sparse.coo_sparse((data, (row, col)), shape=output_shape)

In [ ]:
def coo_kron_gpu(A:cupysparse.coo_matrix, B:cupysparse.coo_matrix):
    out_shape = (A.shape[0] * B.shape[0], A.shape[1] * B.shape[1])

    if A.nnz == 0 or B.nnz == 0:
        # kronecker product is the zero matrix
        return cupysparse.coo_matrix(out_shape).asformat(format)

    # expand entries of A into blocks
    row = A.row.astype(cupy.int32, copy=True) * B.shape[0]
    row = row.repeat(B.nnz)
    col = A.col.astype(cupy.int32, copy=True) * B.shape[1]
    col = col.repeat(B.nnz)
    data = A.data.repeat(B.nnz) 

    # increment block indices
    row = row.reshape(-1, B.nnz)
    row += B.row
    row = row.ravel()

    col = col.reshape(-1, B.nnz)
    col += B.col
    col = col.ravel()

    # compute block entries
    data = data.reshape(-1, B.nnz) * B.data
    data = data.ravel()

    return cupysparse.coo_matrix((data, (row, col)), shape=out_shape).asformat(format)

## Conclusion

Using these improvements, the unitary matrices are roughly 16.000 times more memory efficient and the matrices are generated roughly 2000x times faster.

The largerest circuit tested had 31 qubits on a 192GB of memory system.

## Other approach

This is a German paper where they explain how to build a scaleable GPU-enabled quantum simulator for sciencetific purpuses. It does not use sparse matrices

Willsch, D., Willsch, M., Jin, F., Michielsen, K. and De Raedt, H. (2022). GPU-accelerated simulations of quantum annealing and the quantum approximate optimization algorithm. Computer Physics Communications, 278, p.108411. doi:https://doi.org/10.1016/j.cpc.2022.108411.

## Future improvements

1. **Lazy evaluation and caching**: Lazely generated matrices could still be cached. Currently not possible as it is one of the two.
2. **Use disk for caching**: Especially if matrices are too big.
3. **For extremly large matrices could be processed in blocks, while rest is on disk**